# Networks of Harry's House

In [1]:
#IMPORT

import tweepy
import CONFIG
import csv
import os
import json
import numpy as np
import pandas as pd
import time
import itertools

from collections import Counter

In [2]:
path = os.getcwd() + '\HARRY_HOUSE_DATA'
os.chdir(path)
print(os.getcwd())

C:\Users\sarac\OneDrive - Università degli Studi di Milano\UNI 2022\_SOCIALMEDIA\#harryhouse\HARRY_HOUSE_DATA


In [3]:
client = tweepy.Client(bearer_token = CONFIG.BEARER_TOKEN)

In [4]:
tweets_extended = json.load(open('tweets_extended_1705.json'))
authors = json.load(open('author_1705.json'))
hashtags = json.load(open('hashtags_1705.json'))

## Rete sugli Hashtags

Creando una rete sugli hashtags possiamo individuare quali sono i nodi che trattano degli stessi argomenti. 

Iniziamo con il creare la rete prendendo in considerazione tutti gli hashtag di interesse tranne su _#HarrysHouse_ su cui abbiamo basato la collezione dei dati.

In [5]:
#dizionario in cui le chiavi sono gli hashtage e ci associamo tutti coloro che l'hanno usato
#per ora è solo la struttura

hashtags_used_by_authors = {}
for h in hashtags.keys():
    #non inseriamo HarrysHouse
    if h.lower() != 'harryshouse':
        hashtags_used_by_authors[h.lower()] = []

In [6]:
len(hashtags_used_by_authors)

505

In [7]:
#riempiamo il dizionario
for t in tweets_extended:
    try:
        tweet_entities = t['entities']
    except:
        continue

    list_hashtags = tweet_entities.get('hashtags') #prendo la lista di # usati dall'autore
    author_tweet = t['author_id'] #prendo l'id dell'autore
        
    if list_hashtags:
        for hashtag in list_hashtags:
            h = hashtag['tag'].lower()
            if h != 'harryshouse':
                try:
                    list_users = hashtags_used_by_authors[h]
                except:
                    #è possibile che non sia presente un # nel dict perchè ho tolto i # usati una sola volta
                    continue
                if author_tweet in list_users:
                    #se l'utente è gia stato inserito per un tweet passato non lo inserisco
                    continue
                else:
                    list_users.append(author_tweet)

In [8]:
hashtags_used_by_authors

{'harrystyles': ['1255392567253925892',
  '1441503967922597889',
  '23541990',
  '716687059902136320',
  '1526528647951155200',
  '1525668308',
  '1298396364531994636',
  '1507696040',
  '1361681950025605122',
  '1505199492093644803',
  '864465356',
  '954464759235203072',
  '117889228',
  '1520503375506382848',
  '1503471700448624651',
  '45906569',
  '999920311112880128',
  '922868365433167874',
  '723720694559694848',
  '1397159333281079302',
  '1406047166267809797',
  '1379543658337333248',
  '1188355305509412865',
  '1155303714128879616',
  '1519551985145434117',
  '893944492843298816',
  '1265903562770518016',
  '280566468',
  '164024707',
  '785106175423553536',
  '34080203',
  '87428750',
  '1485631461885726725',
  '1561524319',
  '743132919431049216',
  '838188598692294656',
  '2573672132',
  '1505582417628876805',
  '1427940748402597893',
  '1390812979',
  '1387184265054539780',
  '19401778',
  '122204707',
  '2648299332',
  '33008621',
  '565733262',
  '1525599806105784321',

In [9]:
with open('twitter_network_hashtags_harryshouse_small.csv','w', encoding="utf-8") as ft:
    #riga di intestazione nel file per fargli capire che le colonne sono id e peso (per gephi)
    ft.write('Source,Target,Weight\n')
    #per ogni coppia di account
    for a, b in itertools.combinations(hashtags_used_by_authors.keys(),2):        
        common = set(hashtags_used_by_authors[a]).intersection(set(hashtags_used_by_authors[b]))
        if len(common) > 1: #scrivo sul file solo le coppie che hanno più di un hashtag in comune
            ft.write(f'{a},{b},{len(common)}\n')   

In [10]:
common = 0
with open('twitter_network_hashtags_harryshouse_big.csv','w', encoding="utf-8") as ft:
    #riga di intestazione nel file per fargli capire che le colonne sono id e peso (per gephi)
    ft.write('Source,Target,Weight\n')
    #per ogni coppia di account
    for a, b in itertools.combinations(hashtags_used_by_authors.keys(),2):        
        common = set(hashtags_used_by_authors[a]).intersection(set(hashtags_used_by_authors[b]))
        ft.write(f'{a},{b},{len(common)}\n')  

## Rete dei retweet

In [10]:
accounts2retweeters = {}
for a in authors.keys():
    accounts2retweeters[a] = []

In [12]:
for i, t in enumerate(tweets_extended):
    #prendiamo l'id dell'autore del tweet
    account = t['author_id'] 
    #richiedo coloro che hanno retweettato un tweet
    
    if t['public_metrics']['retweet_count'] > 0:
        users = client.get_retweeters(id = t['id']).data
        if users:
            for user in users:
                accounts2retweeters[account].append(user.data['id'])
        time.sleep(13)
    print(i, end=' ')

json.dump(accounts2retweeters, open('retweeters.json', 'w'))

    




0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 

KeyboardInterrupt: 